In [33]:
# -- 필요 데이터 --
import pandas as pd

def bring_and_preprocess_df():
    # -- 상대적으로 개수가 적은 csv 파일 사용 --
    df = pd.read_csv('/content/drive/MyDrive/reduced_twice_bungae_df.csv', encoding='utf-8-sig')
    df['cat_id'] = df['cat_id'].astype(int).astype(str)

    # -- 패션과 관련 없는 카테고리 제거 --
    idx_list = list()
    for idx in range(len(df)):
        if (len(df.loc[idx, 'cat_id']) == 3) | (df.loc[idx, 'cat_id'][0] not in ['3', '4']):
            idx_list.append(idx)

    df = df[~df.index.isin(idx_list)]
    # -- 카테고리당 상품이 1~2개 있을 경우 train_test_split 했을 때 갯수가 너무 적어 안됨, 이후에 수정 필요 --
    df = df[~df['cat_id'].isin(['310060999', '310220999', '320180500'])]

    df = df.dropna(axis=0)
    df = df.reset_index(drop=True)
    df = df.iloc[:, 2:]
    return df

# 축소 데이터
df = bring_and_preprocess_df()
df['main_cat'] = df['cat_id'].apply(lambda x : x[:3]) # main_cat으로 분리해보기
df['mid_cat'] = df['cat_id'].apply(lambda x : x[:6]) # mid_cag으로도 분리하기
df.shape

(100169, 7)

In [35]:
df.head()

,product_id,product_name,image_url,image_cnt,cat_id,main_cat,mid_cat
0,217365207,경량롱패딩,https://media.bunjang.co.kr/product/217365207_...,3.0,310090050,310,310090
1,175507846,롱패딩 판매합니다,https://media.bunjang.co.kr/product/175507846_...,4.0,310090050,310,310090
2,213094512,피레넥스 여성 화이트 패딩 55판-69,https://media.bunjang.co.kr/product/213094512_...,10.0,310090050,310,310090
3,208884568,보브 다운패딩,https://media.bunjang.co.kr/product/208884568_...,2.0,310090050,310,310090
4,164395818,에이글 코트타입 구스 자켓 L사이즈,https://media.bunjang.co.kr/product/164395818_...,5.0,310090050,310,310090


In [26]:
# -- folder 만들기 --
import os


def make_folders_for_image():
    # -- 임시방편으로 이렇게 만들기 --
    cat_list = list(df['cat_id'].unique())

    for cat_id in cat_list:
        # -- 폴더 이름은 원하시는 대로 변경해주세요 --
        # -- bungae_fashion_image 부분 변경해주시면 돼요 --
        os.makedirs(f'/content/drive/MyDrive/bungae_fashion_image/{cat_id}')

    return

make_folders_for_image()

In [ ]:
# -- making image crawler with multiprocessing ----------

In [25]:
import re

df['image_url'][:5]

0    https://media.bunjang.co.kr/product/217365207_...
1    https://media.bunjang.co.kr/product/175507846_...
2    https://media.bunjang.co.kr/product/213094512_...
3    https://media.bunjang.co.kr/product/208884568_...
4    https://media.bunjang.co.kr/product/164395818_...
Name: image_url, dtype: object

In [ ]:
# -- multiprocessing을 위해 리스트로 url과 cat_id 값 묶기 --
from tqdm.notebook import tqdm

url_cat_id_list = list()
for idx in tqdm(range(len(df))):
    url_cat_id_list.append('__'.join([df.loc[idx, 'image_url'], df.loc[idx, 'cat_id']]))

url_cat_id_list

In [19]:
# -- product_id 부분 --
# 폴더 안에 넣어줄 때 {prd_id}_image.jpg로 넣어줄 예정
prd_id = re.findall(r'\b\/\w+\_', url_cat_id_list[10])[0]
prd_id = prd_id.replace('/', '').replace('_', '')
prd_id

'218095401'

In [12]:
# -- cat_id 부분 --
# 이미지 파일 경로를 설정할 때 {cat_id}/{prd_id}.. 이렇게 설정할 예정
url_cat_id_list[10].split('__')[-1]

'310090050'

In [20]:
# -- multiprocessing을 활용할 수 있도록 리스트로 만들어 txt 파일로 저장 --
with open('/content/drive/MyDrive/bungae_url_cat_id_list.txt', 'w+') as file:
    file.write(','.join(url_cat_id_list))

In [22]:
# -- list 불러오는 방법 --
with open('/content/drive/MyDrive/bungae_url_cat_id_list.txt', 'r') as file:
    data = file.read()
url_cat_id_list = data.split(',') # 상품 list
url_cat_id_list

In [ ]:
# -- crawling py 만들기 ---------------

In [30]:
url_cat_id_list[1]

'https://media.bunjang.co.kr/product/175507846_{cnt}_1641710657_w{res}.jpg__310090050'

In [32]:
# -- base code --

import urllib.request
import time
import pandas as pd
import re
from PIL import Image

def crawl_images_and_put_in_folders(url_and_cat_id):

    removed_prd_id_list = list()

    start = time.time()

    # cat_id
    cat_id = url_and_cat_id.split('__')[-1]

    # url
    url = url_and_cat_id.split('__')[0]

    # prd_id
    prd_id = re.findall(r'\b\/\w+\_', url_and_cat_id)[-1]
    prd_id = prd_id.replace('/', '').replace('_', '')

    try:
        urllib.request.urlretrieve(url, f'/content/drive/MyDrive/bungae_fashion_image/{cat_id}/{prd_id}_image.jpg')

        # image 사이즈 줄이기
        image_path = f'/content/drive/MyDrive/bungae_fashion_image/{cat_id}/{prd_id}_image.jpg'
        image = Image.open(image_path)
        image = image.resize(target_size)
        # Save the resized image
        image.save(image_path)

        end = time.time()
        print(f'{prd_id} image crawled')
    except:
        removed_prd_id_list.append(prd_id)
        pass

    end = time.time()

    return

crawl_images_and_put_in_folders(url_cat_id_list[1])

In [ ]:
# -- moved to vscode --